워드 클라우드 - SOYNLP version

In [1]:
import re, joblib

In [2]:
# 데이터 불러오기
with open('data/gift.txt', encoding='utf-8') as file:
    gift_text = file.read()

In [3]:
gift_text[:100]

'\n반지!목걸이!!!!!!!!\n커플 가디건이나 커플 모자는 어떠신지요 :-)\n몇개 추천제품들 투척하고 갑니다 ㅎㅎ\n더클래식 남녀 커플룩 오버핏 빅사이즈 하프 니트 가디건\nCOUPAN'

In [ ]:
# 데이터 전처리
re.sub('[^가-힣]')